# Collaborative filtering
-----

In this example, we'll build a quick explicit feedback recommender system: that is, a model that takes into account explicit feedback signals (like ratings) to recommend new content.

We'll use an approach first made popular by the [Netflix prize](http://www.netflixprize.com/) contest: [matrix factorization](https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf). 

The basic idea is very simple:

1. Start with user-item-rating triplets, conveying the information that user _i_ gave some item _j_ rating _r_.
2. Represent both users and items as high-dimensional vectors of numbers. For example, a user could be represented by `[0.3, -1.2, 0.5]` and an item by `[1.0, -0.3, -0.6]`.
3. The representations should be chosen so that, when we multiplied together (via [dot products](https://en.wikipedia.org/wiki/Dot_product)), we can recover the original ratings.
4. The utility of the model then is derived from the fact that if we multiply the user vector of a user with the item vector of some item they _have not_ rated, we hope to obtain a predicition for the rating they would have given to it had they seen it.

![collaborative filtering](http://ampcamp.berkeley.edu/big-data-mini-course/img/matrix_factorization.png)
source:[ampcamp.berkeley](http://ampcamp.berkeley.edu/big-data-mini-course/movie-recommendation-with-mllib.html)

## 1. Preparations

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os.path as op

from zipfile import ZipFile
try:
    from urllib.request import urlretrieve
except ImportError:  # Python 2 compat
    from urllib import urlretrieve

# this line needs to be modified if not on colab:
data_folder = '/content/'

ML_100K_URL = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
ML_100K_FILENAME = op.join(data_folder,ML_100K_URL.rsplit('/', 1)[1])
ML_100K_FOLDER = op.join(data_folder,'ml-100k')

We start with importing a famous dataset, the [Movielens 100k dataset](https://grouplens.org/datasets/movielens/100k/). It contains 100,000 ratings (between 1 and 5) given to 1682 movies by 943 users:

In [0]:
if not op.exists(ML_100K_FILENAME):
    print('Downloading %s to %s...' % (ML_100K_URL, ML_100K_FILENAME))
    urlretrieve(ML_100K_URL, ML_100K_FILENAME)

if not op.exists(ML_100K_FOLDER):
    print('Extracting %s to %s...' % (ML_100K_FILENAME, ML_100K_FOLDER))
    ZipFile(ML_100K_FILENAME).extractall(data_folder)

Other datasets, see: [Movielens](https://grouplens.org/datasets/movielens/)

## 2. Data analysis and formating

[Python Data Analysis Library](http://pandas.pydata.org/)

In [0]:
import pandas as pd

all_ratings = pd.read_csv(op.join(ML_100K_FOLDER, 'u.data'), sep='\t',
                          names=["user_id", "item_id", "ratings", "timestamp"])
all_ratings.head()

Let's check out a few macro-stats of our dataset

In [0]:
list_movies_names = []
list_item_ids = []
with open(op.join(ML_100K_FOLDER, 'u.item'), encoding = "ISO-8859-1") as fp:
    for line in fp:
        list_item_ids.append(line.split('|')[0])
        list_movies_names.append(line.split('|')[1])
        
movies_names = pd.DataFrame(list(zip(list_item_ids, list_movies_names)), 
               columns =['item_id', 'item_name']) 
movies_names.head()

In [0]:
movies_names['item_id']=movies_names['item_id'].astype(int)
all_ratings['item_id']=all_ratings['item_id'].astype(int)

In [0]:
all_ratings = all_ratings.merge(movies_names,on='item_id')
all_ratings.head()

In [0]:
#number of entries
len(all_ratings)

In [0]:
all_ratings['ratings'].describe()

In [0]:
# number of unique rating values
len(all_ratings['ratings'].unique())

In [0]:
all_ratings['user_id'].describe()

In [0]:
# number of unique users
total_user_id = len(all_ratings['user_id'].unique())
print(total_user_id)

In [0]:
all_ratings['item_id'].describe()

In [0]:
# number of unique rated items
total_item_id = len(all_ratings['item_id'].unique())
print(total_item_id)

In [0]:
all_ratings['item_id'] = all_ratings['item_id'].apply(lambda x :x-1)
all_ratings['user_id'] = all_ratings['user_id'].apply(lambda x :x-1)

In [0]:
movies_names['item_id']=movies_names['item_id'].apply(lambda x: x-1)

In [0]:
movies_names=movies_names.set_index('item_id')

In [0]:
movies_names.head()

For spliting the data into _train_ and _test_ we'll be using a pre-defined function from [scikit-learn](http://scikit-learn.org/stable/)

In [0]:
from sklearn.model_selection import train_test_split

ratings_train, ratings_test = train_test_split(
    all_ratings, test_size=0.2, random_state=42)

user_id_train = ratings_train['user_id']
item_id_train = ratings_train['item_id']
rating_train = ratings_train['ratings']

user_id_test = ratings_test['user_id']
item_id_test = ratings_test['item_id']
rating_test = ratings_test['ratings']

In [0]:
len(user_id_train)

In [0]:
len(user_id_train.unique())

In [0]:
len(item_id_train.unique())

We see that all the movies are not rated in the train set.

In [0]:
movies_not_train = (set(all_ratings['item_id']) -set(item_id_train))
for m in movies_not_train:
    print(m,movies_names.loc[m]['item_name'])

In [0]:
user_id_train.iloc[:5]

In [0]:
item_id_train.iloc[:5]

In [0]:
rating_train.iloc[:5]

## 3. The model

We can feed our dataset to the `FactorizationModel` class - a sklearn-like object that allows us to train and evaluate the explicit factorization models.

Internally, the model uses the `Model_dot`(class to represents users and items. It's composed of a 4 `embedding` layers:

- a `(num_users x latent_dim)` embedding layer to represent users,
- a `(num_items x latent_dim)` embedding layer to represent items,
- a `(num_users x 1)` embedding layer to represent user biases, and
- a `(num_items x 1)` embedding layer to represent item biases.

In [0]:
import torch.nn as nn
import torch

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Let's generate [Embeddings](http://pytorch.org/docs/master/nn.html#embedding) for the users, _i.e._ a fixed-sized vector describing the user

In [0]:
embedding_dim = 3
embedding_user = nn.Embedding(total_user_id, embedding_dim)
input = torch.LongTensor([[1,2,4,5],[4,3,2,0]])
embedding_user(input)

We will use some custom embeddings and dataloader

In [0]:
class ScaledEmbedding(nn.Embedding):
    """
    Embedding layer that initialises its values
    to using a normal variable scaled by the inverse
    of the emedding dimension.
    """
    def reset_parameters(self):
        """
        Initialize parameters.
        """
        self.weight.data.normal_(0, 1.0 / self.embedding_dim)
        if self.padding_idx is not None:
            self.weight.data[self.padding_idx].fill_(0)


class ZeroEmbedding(nn.Embedding):
    """
    Used for biases.
    """
    def reset_parameters(self):
        """
        Initialize parameters.
        """
        self.weight.data.zero_()
        if self.padding_idx is not None:
            self.weight.data[self.padding_idx].fill_(0)

In [0]:
class DotModel(nn.Module):
    
    def __init__(self,
                 num_users,
                 num_items,
                 embedding_dim=32):
        
        super(DotModel, self).__init__()
        
        self.embedding_dim = embedding_dim
        
        self.user_embeddings = ScaledEmbedding(num_users, embedding_dim)
        self.item_embeddings = ScaledEmbedding(num_items, embedding_dim)
        self.user_biases = ZeroEmbedding(num_users, 1)
        self.item_biases = ZeroEmbedding(num_items, 1)
                
        
    def forward(self, user_ids, item_ids):
        
        #
        # your code here
        #
        return 


In [0]:
net = DotModel(total_user_id,total_item_id).to(device)

Now test your network on a small batch.

In [0]:
batch_users_np = user_id_train.values[:5].astype(np.int32)
batch_items_np = item_id_train.values[:5].astype(np.int32)
batch_ratings_np = rating_train[:5].values.astype(np.float32)
batch_users_tensor = torch.LongTensor(batch_users_np).to(device)
batch_items_tensor = torch.LongTensor(batch_items_np).to(device)
batch_ratings_tensor = torch.tensor(batch_ratings_np).to(device)

In [0]:
predicitions = net(batch_users_tensor,batch_items_tensor)

In [0]:
predicitions

We will use MSE loss defined below:

In [0]:
def regression_loss(predicted_ratings, observed_ratings):
    return ((observed_ratings - predicted_ratings) ** 2).mean()

In [0]:
loss_fn = regression_loss
loss = loss_fn(predicitions, batch_ratings_tensor)

In [0]:
loss

Check that your network is learning by overfitting your network on this small batch (you should reach a loss below 0.5 in the cell below).

In [0]:
net = DotModel(total_user_id,total_item_id).to(device)
optimizer = torch.optim.Adam(net.parameters(), lr = 0.1)
for e in range(15):
    #
    # your code here
    #

In [0]:
def shuffle(*arrays):

    random_state = np.random.RandomState()
    shuffle_indices = np.arange(len(arrays[0]))
    random_state.shuffle(shuffle_indices)

    if len(arrays) == 1:
        return arrays[0][shuffle_indices]
    else:
        return tuple(x[shuffle_indices] for x in arrays)

In [0]:
def minibatch(batch_size, *tensors):

    if len(tensors) == 1:
        tensor = tensors[0]
        for i in range(0, len(tensor), batch_size):
            yield tensor[i:i + batch_size]
    else:
        for i in range(0, len(tensors[0]), batch_size):
            yield tuple(x[i:i + batch_size] for x in tensors)



In [0]:
import imp
import numpy as np

import torch.optim as optim

class FactorizationModel(object):
    
    def __init__(self, embedding_dim=32, n_iter=10, batch_size=256, l2=0.0,
                 learning_rate=1e-2, device=device, net=None, num_users=None,
                 num_items=None,random_state=None):
        
        self._embedding_dim = embedding_dim
        self._n_iter = n_iter
        self._learning_rate = learning_rate
        self._batch_size = batch_size
        self._l2 = l2
        self._device = device
        self._num_users = num_users
        self._num_items = num_items
        self._net = net
        self._optimizer = None
        self._loss_func = None
        self._random_state = random_state or np.random.RandomState()
             
        
    def _initialize(self):
        if self._net is None:
            self._net = DotModel(self._num_users, self._num_items, self._embedding_dim).to(self._device)
        
        self._optimizer = optim.Adam(
                self._net.parameters(),
                lr=self._learning_rate,
                weight_decay=self._l2
            )
        
        self._loss_func = regression_loss
    
    @property
    def _initialized(self):
        return self._optimizer is not None
    
    
    def fit(self, user_ids, item_ids, ratings, verbose=True):
        
        user_ids = user_ids.astype(np.int64)
        item_ids = item_ids.astype(np.int64)
        
        if not self._initialized:
            self._initialize()
            
        for epoch_num in range(self._n_iter):
            users, items, ratingss = shuffle(user_ids,
                                            item_ids,
                                            ratings)

            user_ids_tensor = torch.from_numpy(users).to(self._device)
            item_ids_tensor = torch.from_numpy(items).to(self._device)
            ratings_tensor = torch.from_numpy(ratingss).to(self._device)
            epoch_loss = 0.0

            for (minibatch_num,
                 (batch_user,
                  batch_item,
                  batch_rating)) in enumerate(minibatch(self._batch_size,
                                                         user_ids_tensor,
                                                         item_ids_tensor,
                                                         ratings_tensor)):
                
                
                # beging to be completed
                predictions = 
                #
                loss = 
                epoch_loss = 
                #
                #
                # end to be completed
            
            epoch_loss = epoch_loss / (minibatch_num + 1)
            
            if verbose:
                print('Epoch {}: loss_train {}'.format(epoch_num, epoch_loss))
        
            if np.isnan(epoch_loss) or epoch_loss == 0.0:
                raise ValueError('Degenerate epoch loss: {}'
                                 .format(epoch_loss))
    
    
    def test(self,user_ids, item_ids, ratings):
        self._net.train(False)
        user_ids = user_ids.astype(np.int64)
        item_ids = item_ids.astype(np.int64)
        
        user_ids_tensor = torch.from_numpy(user_ids).to(self._device)
        item_ids_tensor = torch.from_numpy(item_ids).to(self._device)
        ratings_tensor = torch.from_numpy(ratings).to(self._device)
               
        predictions = self._net(user_ids_tensor, item_ids_tensor)
        
        loss = self._loss_func(ratings_tensor, predictions)
        return loss.data.item()

    def predict(self,user_ids, item_ids):
        self._net.train(False)
        user_ids = user_ids.astype(np.int64)
        item_ids = item_ids.astype(np.int64)
        
        user_ids_tensor = torch.from_numpy(user_ids).to(self._device)
        item_ids_tensor = torch.from_numpy(item_ids).to(self._device)
               
        predictions = self._net(user_ids_tensor, item_ids_tensor)
        return predictions.data   

In [0]:
model = FactorizationModel(embedding_dim=50,  # latent dimensionality
                                   n_iter=5,  # number of epochs of training
                                   batch_size=1024,  # minibatch size
                                   learning_rate=1e-3,
                                   l2=1e-9,  # strength of L2 regularization
                                   num_users=total_user_id,
                                   num_items=total_item_id)

In [0]:
user_ids_train_np = user_id_train.values.astype(np.int32)
item_ids_train_np = item_id_train.values.astype(np.int32)
ratings_train_np = rating_train.values.astype(np.float32)
user_ids_test_np = user_id_test.values.astype(np.int64)
item_ids_test_np = item_id_test.values.astype(np.int64)
ratings_test_np = rating_test.values.astype(np.float32)

In [0]:
model.fit(user_ids_train_np, item_ids_train_np, ratings_train_np)

In [0]:
model.test(user_ids_test_np, item_ids_test_np, ratings_test_np  )

In [0]:
print(model._net)

In [0]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

test_preds = model.predict(user_ids_test_np, item_ids_test_np)
print("Final test RMSE: %0.3f" % np.sqrt(mean_squared_error(test_preds.cpu(), ratings_test_np)))
print("Final test MAE: %0.3f" % mean_absolute_error(test_preds.cpu(), ratings_test_np))

You can compare with [Surprise](https://github.com/NicolasHug/Surprise)

## 4. Best and worst movies

Getting the name of the movies (there must be a better way, please provide alternate solutions!)

In [0]:
list_movies_names = []
list_item_ids = []
with open(op.join(ML_100K_FOLDER, 'u.item'), encoding = "ISO-8859-1") as fp:
    for line in fp:
        list_item_ids.append(line.split('|')[0])
        list_movies_names.append(line.split('|')[1])
        
movies_names = pd.DataFrame(list(zip(list_item_ids, list_movies_names)), 
               columns =['item_id', 'item_name']) 
movies_names.head()

In [0]:
item_bias_np = model._net.item_biases.weight.data.cpu().numpy()

In [0]:
movies_names['biases'] = pd.Series(item_bias_np.T[0], index=movies_names.index)

In [0]:
movies_names.head()

In [0]:
movies_names.shape

In [0]:
indices_item_train = np.sort(item_id_train.unique())
movies_names = movies_names.loc[indices_item_train]
movies_names.shape

In [0]:
movies_names = movies_names.sort_values(ascending=False,by=['biases'])

Best movies

In [0]:
movies_names.head(10)

Worse movies

In [0]:
movies_names.tail(10)

## 5. PCA

In [0]:
item_emb_np = model._net.item_embeddings.weight.data.cpu().numpy()
item_emb_np.shape

In [0]:
from sklearn.decomposition import PCA
from operator import itemgetter

pca = PCA(n_components=3)
latent_fac = pca.fit_transform(item_emb_np)

In [0]:
movie_comp = [(f, i) for f,i in zip(latent_fac[:,1], list_movies_names)]

In [0]:
sorted(movie_comp, key=itemgetter(0), reverse=True)[:10]

In [0]:
sorted(movie_comp, key=itemgetter(0), reverse=False)[:10]

In [0]:
g = all_ratings.groupby('item_name')['ratings'].count()
most_rated_movies = g.sort_values(ascending=False).index.values[:1000]
most_rated_movies[:10]

In [0]:
idxs = range(50)
txt_movies_names = most_rated_movies[:len(idxs)]
X = latent_fac[idxs,0]
Y = latent_fac[idxs,2]
plt.figure(figsize=(15,15))
plt.scatter(X, Y)
for i, x, y in zip(txt_movies_names, X, Y):
    plt.text(x+0.01,y-0.01,i, fontsize=11)
plt.show()

## 6. SPOTLIGHT

The code written above is a simplified version of [SPOTLIGHT](https://github.com/maciejkula/spotlight)

Once you installed it with: `conda install -c maciejkula -c pytorch spotlight=0.1.5`, you can compare the results...